# Contextual Precision

The contextual precision metric uses LLM-as-a-judge to measure your RAG pipeline's retriever by evaluating whether nodes in your retrieval_context that are relevant to the given input are ranked higher than irrelevant ones. deepeval's contextual precision metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.


In [1]:
import os
import wikipedia
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from IPython.display import display, Markdown
from pathlib import Path
from dotenv import load_dotenv

from deepeval import evaluate
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.test_case import LLMTestCase

In [2]:
dotenv_path = Path("../../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model_eval = ChatOpenAI(model="gpt-4o-mini")

In [4]:
enc = tiktoken.get_encoding("o200k_base")

## Recovering the context (from wikipedia)

In [5]:
wikipedia.set_lang("en")
page = wikipedia.page("Albert Einstein")
context = page.content
display(Markdown(f"**Context:**"))
display(Markdown(f"----"))
display(Markdown(f"{context[:500]}...{context[-500:]}"))
display(Markdown(f"----"))
display(Markdown(f"Estimated number of tokens: {len(enc.encode(context))}"))

**Context:**

----

Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric eff...in
Finding aid to Albert Einstein Collection from Center for Jewish History


==== Digital collections ====
The Digital Einstein Papers — An open-access site for The Collected Papers of Albert Einstein, from Princeton University
Albert Einstein Digital Collection from Vassar College Digital Collections
Newspaper clippings about Albert Einstein in the 20th Century Press Archives of the ZBW
Albert – The Digital Repository of the IAS, which contains many digitized original documents and photographs

----

Estimated number of tokens: 17617

## Question

In [8]:
chat = ChatOllama(model="llama3.2")

prompt = f"""
In what year was the German scientist Albert Einstein born?
"""

response = chat.invoke(prompt)
answer = response.content

display(Markdown(f"**Prompt:**"))
display(Markdown(f"----"))
display(Markdown(prompt))
display(Markdown(f"----"))

display(Markdown(f"**Answer:**"))
display(Markdown(f"----"))
display(Markdown(f"{answer}"))
display(Markdown(f"----"))

**Prompt:**

----


In what year was the German scientist Albert Einstein born?


----

**Answer:**

----

Albert Einstein was not a German scientist, he was actually born in Germany (then known as the Kingdom of Württemberg, part of the German Empire) to Swiss parents. He moved to Switzerland at a young age and later became a Swiss citizen.

However, I think you may be confusing him with another famous scientist. Albert Einstein is widely recognized for his groundbreaking work in theoretical physics, particularly his theory of relativity.

To clarify, Albert Einstein was born on March 14, 1879, in Ulm, Kingdom of Württemberg, German Empire (now part of modern-day Germany).

----

## Contextual Precision evaluation

In [9]:
metric = ContextualPrecisionMetric(
    model="gpt-4o-mini",
    include_reason=True)

test_case = LLMTestCase(
    input=prompt,
    actual_output=answer,
    expected_output="1879",
    retrieval_context=[context]
)

metric.measure(test_case)

display(Markdown(f"**Eval:**"))
display(Markdown(f"----"))
display(Markdown(f"**Grade**: {metric.score}"))
display(Markdown(f"----"))
display(Markdown(f"**Reason**: {metric.reason}"))
display(Markdown(f"----"))

C:\Users\leona\Documents\GitHub\deepeval_studies\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

**Eval:**

----

**Grade**: 1.0

----

**Reason**: The score is 1.00 because the first node provides a direct answer to the question about Albert Einstein's birth year, stating that he was born in 1879. There are no irrelevant nodes present, which contributes to the perfect score.

----